# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [3]:
import pandas as pd
import nltk
from collections import Counter
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()
df = pd.read_csv('/datasets/credit_scoring_eng.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


- There are over 21,000 rows but only missing data in total_income and days_employed
- Most columns seem to have adequate data types but floats would be more useful as ints


### Step 2. Data preprocessing

### Processing missing values

In [2]:
df['days_employed'].sort_values()
#checked to see if numbers are all positive. They are not. 
days_employed_positive_mean = df['days_employed'][df['days_employed'] > 0].mean()
days_employed_positive_median = df['days_employed'][df['days_employed'] > 0].median()
print('Days_Employed Mean:', days_employed_positive_mean, 'vs. Median:', days_employed_positive_median)
#checking to see if there are any major outliers in the positive data but both numbers are roughly the same.   
df['days_employed'] = df['days_employed'].fillna(days_employed_positive_mean)

#doing the same process for total_income
df['total_income'].sort_values()
#all values are positive
total_income_mean = df['total_income'].mean()
total_income_median = df['total_income'].median()
print('Total Income Mean:', total_income_mean, 'vs. Median:', total_income_median)
#again, both numbers are roughly the same.  Using smaller number. 
df['total_income'] = df['total_income'].fillna(total_income_median)


#checking to make sure there are no more NaN values
df.isnull().sum()

Days_Employed Mean: 365004.3099162686 vs. Median: 365213.30626573117
Total Income Mean: 26787.56835465867 vs. Median: 23202.87


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Conclusion

- It looks like the same rows that were missing days of employment were also missing total income.  This likely means this information was not required at the time those specific rows of data were collected.  
- Unfortunately we have negative values in the days_employed data.  This is obviously an error that will need to be fixed before using this number to draw any conclusions. 

### Data type replacement

In [3]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
#double checking there are no more floats
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Conclusion

- Used the astype method since tonumeric method only changes input to floats

### Processing duplicates

In [4]:
dupedf = df[df.duplicated()]
#checking for duplicates, sorting by dob_years to double check there are actually duplicates. 
print(dupedf.sort_values('dob_years'))
dupedf['dob_years'].count()

       children  days_employed  dob_years            education  education_id  \
19321         0         365004         23  secondary education             1   
18328         0         365004         29    bachelor's degree             0   
21281         1         365004         30    bachelor's degree             0   
18349         1         365004         30    bachelor's degree             0   
13878         1         365004         31  secondary education             1   
16904         1         365004         32    bachelor's degree             0   
4182          1         365004         34    BACHELOR'S DEGREE             0   
9238          2         365004         34  secondary education             1   
13773         0         365004         35  secondary education             1   
14432         2         365004         36    bachelor's degree             0   
12375         1         365004         37  secondary education             1   
19387         0         365004         3

54

### Conclusion

I struggled with this section quite a bit.  When using df.duplicated(), it would return 54 values, but upon closer look of those 54 values, they were actually not total duplicates. For example, the list of 54 returned only 2 people who are 30 and have a child but their purpose and income type are not the same.  As I did not find any duplicates, I did not delete any row from the dataframe

### Categorizing Data

In [5]:
#splitting data by income level
def income_level(income):
    if income <= 20000:
        return 'low'
    if income <= 50000:
        return 'mid low'
    if income <= 70000:
        return 'mid high'
    else:
        return 'high'
    
df['income_level'] = df['total_income'].apply(income_level)


### Conclusion

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [8]:
total_with_children = df['children'][(df['children'] != 0)].count()
print('Applicants without children', total_with_children)
df_children_debt = df['children'][(df['children'] != 0) & (df['debt'] == 1)].count()
ratio_with_children = df_children_debt / total_with_children
print('Liklihood of missed payments with children: {:.2%}'.format(ratio_with_children))

total_without_children = df['children'][(df['children'] == 0)].count()
print('Applicants without children', total_without_children)
df_nochildren_debt = df['children'][(df['children'] == 0) & (df['debt'] == 1)].count()
ratio_without_children = df_nochildren_debt / total_without_children
print('Liklihood of missed payments without children: {:.2%}'.format(ratio_without_children))

Applicants without children 7376
Liklihood of missed payments with children: 9.19%
Applicants without children 14149
Liklihood of missed payments without children: 7.51%


### Conclusion

- From the data, it looks like that a person is more likely to default if they have children.  
- Additionally, there are twice as many people without children which is also interesting to consider. 

- Is there a relation between marital status and repaying a loan on time?

In [9]:
total_married = df['family_status'][(df['family_status'] == 'married')].count()
print('Married applicants', total_married)
df_married_debt = df['family_status'][(df['family_status'] == 'married') & (df['debt'] == 1)].count()
ratio_married = df_married_debt / total_married
print('Liklihood of missed payments when married: {:.2%}'.format(ratio_married))

total_not_married = df['family_status'][(df['family_status'] != 'married')].count()
print('Applicants not in a marriage', total_not_married)
df_not_married_debt = df['family_status'][(df['family_status'] != 'married') & (df['debt'] == 1)].count()
ratio_not_married = df_not_married_debt / total_not_married
print('Liklihood of missed payments when not married: {:.2%}'.format(ratio_not_married))
print()
print('Breakdown of those not married:')
total_CP = df['family_status'][(df['family_status'] == 'civil partnership')].count()
df_CP_debt = df['family_status'][(df['family_status'] == 'civil partnership') & (df['debt'] == 1)].count()
ratio_CP = df_CP_debt / total_CP
print('Liklihood of missed payments when in a civil partnership: {:.2%}'.format(ratio_CP))

total_unmarried = df['family_status'][(df['family_status'] == 'unmarried')].count()
df_unmarried_debt = df['family_status'][(df['family_status'] == 'unmarried') & (df['debt'] == 1)].count()
ratio_unmarried = df_unmarried_debt / total_unmarried
print('Liklihood of missed payments when unmarried: {:.2%}'.format(ratio_unmarried))

total_divorced = df['family_status'][(df['family_status'] == 'divorced')].count()
df_divorced_debt = df['family_status'][(df['family_status'] == 'divorced') & (df['debt'] == 1)].count()
ratio_divorced = df_divorced_debt / total_divorced
print('Liklihood of missed payments when divorced: {:.2%}'.format(ratio_divorced))

total_widow = df['family_status'][(df['family_status'] == 'widow / widower')].count()
df_widow_debt = df['family_status'][(df['family_status'] == 'widow / widower') & (df['debt'] == 1)].count()
ratio_widow = df_widow_debt / total_widow
print('Liklihood of missed payments when widowed: {:.2%}'.format(ratio_widow))

Married applicants 12380
Liklihood of missed payments when married: 7.52%
Applicants not in a marriage 9145
Liklihood of missed payments when not married: 8.86%

Breakdown of those not married:
Liklihood of missed payments when in a civil partnership: 9.29%
Liklihood of missed payments when unmarried: 9.74%
Liklihood of missed payments when divorced: 7.11%
Liklihood of missed payments when widowed: 6.56%


### Conclusion

- When comparing between those who are married and everyone else, those who are married are less likely to not pay their loan. 
- However, within those who are not married, the percentage varies greatly.  It seems that divorced and widowed people are the safest bet. 
- Of course, this should not be the only factor when assessing loan eligibility

- Is there a relation between income level and repaying a loan on time?

In [10]:
total_low = df['income_level'][(df['income_level'] == 'low')].count()
df_low_debt = df['income_level'][(df['income_level'] == 'low') & (df['debt'] == 1)].count()
ratio_low_income = df_low_debt / total_low
print('Liklihood of missed payments with low income: {:.2%}'.format(ratio_low_income))

total_med_low = df['income_level'][(df['income_level'] == 'mid low')].count()
df_med_low_debt = df['income_level'][(df['income_level'] == 'mid low') & (df['debt'] == 1)].count()
ratio_med_low_income = df_med_low_debt / total_med_low
print('Liklihood of missed payments with mid-low income: {:.2%}'.format(ratio_med_low_income))

total_med_high = df['income_level'][(df['income_level'] == 'mid high')].count()
df_med_high_debt = df['income_level'][(df['income_level'] == 'mid high') & (df['debt'] == 1)].count()
ratio_med_high_income = df_med_high_debt / total_med_high
print('Liklihood of missed payments with mid-high income: {:.2%}'.format(ratio_med_high_income))

total_high = df['income_level'][(df['income_level'] == 'high')].count()
df_high_debt = df['income_level'][(df['income_level'] == 'high') & (df['debt'] == 1)].count()
ratio_high_income = df_high_debt / total_high
print('Liklihood of missed payments with high income: {:.2%}'.format(ratio_high_income))

Liklihood of missed payments with low income: 8.25%
Liklihood of missed payments with mid-low income: 8.11%
Liklihood of missed payments with mid-high income: 7.43%
Liklihood of missed payments with high income: 5.82%


### Conclusion

- The higher the income, the less likely to miss payments
- Another factor to consider is that 2000+ values were changed that are now in the mid-low level.  If we had the real data, the percentages above may change. However, it is noted that none of the changed values had ever missed payments.  

- How do different loan purposes affect on-time repayment of the loan?

In [11]:
#see value counts of all unique purposes
print(df['purpose'].value_counts())
#many purposes are the same, just stated differently. Finding lemmas of words will help to shorten the list 
all_purpose = ' '.join([i for i in df['purpose']]).split()
lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in all_purpose]
print()
print('Count by lemmas:')
print(Counter(lemmas))
print()
#spliting data by purpose & finding debt ratio

total_house = df[(df['purpose'].str.contains('estate')) | 
                                 (df['purpose'].str.contains('property')) | 
                                 (df['purpose'].str.contains('house')) | 
                                 (df['purpose'].str.contains('housing'))]
house_debt = total_house['debt'][(total_house['debt'] == 1)]
ratio_house_debt = (house_debt.count()) / total_house['debt'].count()
print('Liklihood of missed payments for house: {:.2%}'.format(ratio_house_debt))


total_education = df[(df['purpose'].str.contains('university')) | 
                                 (df['purpose'].str.contains('education')) | 
                                 (df['purpose'].str.contains('educated'))]
education_debt = total_education['debt'][(total_education['debt'] == 1)]
ratio_education_debt = (education_debt.count()) / total_education['debt'].count()
print('Liklihood of missed payments for education: {:.2%}'.format(ratio_education_debt))

total_car = df[(df['purpose'].str.contains('car'))]
car_debt = total_car['debt'][(total_car['debt'] == 1)]
ratio_car_debt = (car_debt.count()) / total_car['debt'].count()
print('Liklihood of missed payments for car: {:.2%}'.format(ratio_car_debt))

total_wedding = df[(df['purpose'].str.contains('wedding'))]
wedding_debt = total_wedding['debt'][(total_wedding['debt'] == 1)]
ratio_wedding_debt = (wedding_debt.count()) / total_wedding['debt'].count()
print('Liklihood of missed payments for wedding: {:.2%}'.format(ratio_wedding_debt))



wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
housing                                     647
purchase of the house                       647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
purchase of my own house                    620
building a property                         620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

### Conclusion

- It appears that those needing the money for education or a car are the most risky. 
- In the future it would be easier to have uniform purposes to choose from, eliminating the need to search manually within the data

### Step 4. General conclusion

From the questions presented, we can assume that low income, unmarried applicants with children looking to buy a car or pay for education are the most risky.  However, other questions could be explored as well that were not presented above.  For instance, does a person's age factor into their ability to pay on time? Or their education, days employed, or income type. 